## Transformer

In [ ]:
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import random
 




In [29]:
import ast
import random

import numpy as np
from numpy import e
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import KFold

root = 'data_processed/'

### Import Dataset

In [3]:
def safe_parse_all_columns_df(df):
    """
    Parse all columns in a DataFrame to numeric, coercing errors.
    """
    df['notes'] = df['notes'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['chords'] = df['chords'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['velocities'] = df['velocities'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['durations'] = df['durations'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['offsets'] = df['offsets'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['ordered_events'] = df['ordered_events'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    return df

def load_dataframe_from_two_csvs(file1, file2):
    """
    Load and concatenate two CSV files into a single pandas DataFrame.
    """
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    full_df = pd.concat([df1, df2], ignore_index=True)
    full_df = safe_parse_all_columns_df(full_df)

    return full_df

def save_dataframe_to_two_csvs(df, file1, file2):
    """
    Split a DataFrame in half and save it into two CSV files.
    """
    halfway = len(df) // 2
    df.iloc[:halfway].to_csv(file1, index=False)
    df.iloc[halfway:].to_csv(file2, index=False)

def load_dataframe_from_one_csv(file):
    """
    Load a DataFrame from a single CSV file.
    """
    df = pd.read_csv(file)
    
    return df

def save_dataframe_to_one_csv(df, file):
    """
    Save a DataFrame to a single CSV file.
    """
    df.to_csv(file, index=True)

def load_reconstructed_events(file):
    """
    Loads the reconstructed events CSV and safely parses the 'sequence' column,
    converting notes to integers and chords to lists of integers.
    """
    df = pd.read_csv(file)

    def safe_parse(seq_str):
        try:
            parsed = ast.literal_eval(seq_str)
            if not isinstance(parsed, list):
                raise ValueError("Parsed sequence is not a list")

            normalized = []
            for el in parsed:
                if isinstance(el, list):
                    normalized.append([int(x) for x in el])
                else:
                    normalized.append(int(el))
            return normalized

        except Exception as e:
            print(f"Error parsing sequence: {seq_str}")
            raise e

    df['sequence'] = df['sequence'].apply(safe_parse)
    return df

In [4]:
file1 = root + 'data_part1.csv'
file2 = root + 'data_part2.csv'

df = load_dataframe_from_two_csvs(file1, file2)

In [5]:
df

,file_name,instrument,track_names,notes,chords,velocities,durations,offsets,tempos,time_signatures,key_signatures,ordered_events
0,0_Begin The Beguine - Live At Maybeck Recital ...,['Unknown'],['Piano'],"[88, 38, 38, 45, 45, 45, 45, 45, 38, 45, 61, 4...","[[44, 45], [60, 55], [44, 45], [65, 38], [70, ...","[51, 35, 52, 48, 48, 50, 50, 50, 49, 42, 56, 4...","[0.25, 2.5, 0.25, 0.3333, 0.3333, 0.25, 0.3333...","[0.5, 1.25, 3.75, 1.6667, 2.3333, 2.75, 3.5, 0...",[120.0],['4/4'],[],"[n, n, n, n, n, n, c, n, n, n, c, c, n, n, n, ..."
1,1000_Please Be Kind.midi,['Unknown'],['Piano'],"[77, 77, 70, 75, 77, 73, 70, 75, 73, 77, 73, 4...","[[56, 65, 68, 80, 60], [54, 61, 65, 75], [75, ...","[49, 55, 55, 56, 63, 54, 46, 55, 56, 59, 60, 6...","[1.25, 1.6667, 0.25, 1.0, 1.0, 0.5, 0.6667, 0....","[0.6667, 2.0, 3.75, 1.0, 2.5, 1.25, 3.3333, 0....",[120.0],['4/4'],[],"[c, c, n, n, n, c, c, n, c, n, n, n, n, c, n, ..."
2,1001_September Song.midi,['Unknown'],['Piano'],"[46, 70, 70, 79, 77, 79, 74, 79, 70, 67, 62, 7...","[[53, 60, 62], [65, 46, 53, 60], [55, 58, 63, ...","[55, 63, 62, 48, 67, 59, 50, 44, 64, 61, 62, 5...","[2.6667, 0.25, 1.25, 0.75, 0.75, 1.6667, 1.75,...","[0.6667, 3.75, 0.6667, 3.25, 1.0, 1.75, 0.25, ...",[120.0],['4/4'],[],"[n, n, c, c, n, c, c, c, n, n, n, c, c, n, n, ..."
3,1002_Tea For Two.midi,['Unknown'],['Piano'],"[64, 95, 52, 65, 69, 71, 69, 83, 81, 53, 93, 7...","[[52, 28, 40], [65, 68], [53, 55, 56], [58, 60...","[72, 75, 58, 60, 60, 60, 63, 67, 74, 72, 66, 6...","[3.75, 2.5, 1.0, 2.0, 1.6667, 1.3333, 0.75, 0....","[0.25, 0.3333, 3.0, 2.0, 2.3333, 2.6667, 3.25,...",[120.0],['4/4'],[],"[c, n, c, c, c, c, c, c, n, c, c, c, c, c, c, ..."
4,1003_The Very Thought Of You.midi,['Unknown'],['Piano'],"[59, 59, 62, 50, 74, 76, 62, 59, 59, 64, 66, 7...","[[60, 62, 38, 54], [43, 62, 50, 52, 57], [43, ...","[65, 60, 48, 64, 44, 39, 46, 47, 47, 55, 53, 5...","[0.75, 0.6667, 0.6667, 0.5, 1.0, 0.25, 2.0, 0....","[0.6667, 1.75, 3.3333, 1.3333, 2.3333, 3.5, 0....",[120.0],['4/4'],[],"[n, n, c, n, c, n, c, c, c, n, n, c, c, n, n, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2770,997_I Don't Stand A Ghost Of A Chance With You...,['Unknown'],['Piano'],"[61, 49, 65, 70, 63, 66, 64, 75, 72, 73, 66, 5...","[[63, 60], [58, 61, 66, 49], [62, 68, 59], [48...","[70, 53, 59, 61, 62, 51, 50, 66, 44, 60, 63, 6...","[1.3333, 1.6667, 1.0, 0.5, 0.5, 0.5, 0.6667, 0...","[0.6667, 2.0, 2.0, 3.25, 2.5, 3.5, 2.75, 1.5, ...",[120.0],['4/4'],[],"[n, n, c, n, n, c, c, n, c, n, n, n, n, n, c, ..."
2771,998_I'll See You In My Dreams.midi,['Unknown'],['Piano'],"[40, 45, 53, 43, 67, 61, 69, 60, 65, 63, 69, 3...","[[45, 55, 67, 69, 72, 75], [46, 74, 56, 62, 68...","[54, 51, 58, 51, 54, 55, 49, 51, 49, 37, 58, 4...","[1.0, 1.0, 0.25, 1.25, 0.3333, 0.25, 0.25, 0.7...","[0.5, 1.75, 3.75, 2.6667, 0.25, 0.75, 1.25, 1....",[120.0],['4/4'],[],"[c, c, c, c, c, c, n, c, n, n, c, n, c, n, c, ..."
2772,999_Lover Come Back To Me.midi,['Unknown'],['Piano'],"[71, 52, 61, 64, 45, 69, 45, 73, 61, 63, 68, 6...","[[45, 57], [45, 69], [69, 45, 57], [66, 54], [...","[60, 68, 57, 63, 65, 50, 69, 57, 43, 67, 47, 4...","[1.6667, 0.6667, 3.3333, 2.75, 2.25, 0.75, 0.5...","[0.6667, 2.75, 0.6667, 0.6667, 0.6667, 3.25, 1...",[120.0],['4/4'],[],"[c, n, n, n, n, n, n, c, n, n, n, c, c, n, n, ..."
2773,99_Dat Dere - Live.midi,['Unknown'],['Piano'],"[48, 41, 60, 66, 43, 55, 61, 36, 48, 41, 60, 6...","[[67, 68, 72, 79], [65, 68, 72, 77], [60, 63, ...","[75, 72, 70, 74, 72, 68, 74, 67, 71, 63, 67, 6...","[0.3333, 0.3333, 0.25, 0.25, 0.3333, 0.3333, 0...","[0.3333, 0.6667, 2.5, 2.75, 3.0, 3.3333, 0.5, ...",[120.0],['4/4'],[],"[n, n, n, c, n, c, n, n, n, c, n, c, n, n, n, ..."


In [6]:
len(df['ordered_events'][0])

2429

In [7]:
len(df['durations'][0])

2429

In [8]:
len(df['notes'][0])

1378

In [9]:
len(df['chords'][0])

1051

### Useful functions

In [10]:
def parse_chord_to_list(chord):
    """
    Convert a chord string to a list of integers.
    """
    if isinstance(chord, str):
        print([int(x) for x in chord.split(',') if x.isdigit()])
        return [int(x) for x in chord.split(',') if x.isdigit()]
    return []

In [ ]:
def reconstruct_ordered_events(df):
    """
    Reconstruct the ordered list of events (notes and chords) for each song.
    """
    sequences  = []

    for i in range(len(df)):
        idx_note = 0
        idx_chord = 0
        reconstructed = []

        for element in df['ordered_events'][i]:
            if element == 'n':
                reconstructed.append(df['notes'][i][idx_note])
                idx_note += 1
            elif element == 'c':
                parsed_chord = parse_chord_to_list(df['chords'][i][idx_chord])
                reconstructed.append(df['chords'][i][idx_chord])
                idx_chord += 1
            else:
                raise ValueError(f"Unknown event type: {e}")
        
        sequences.append(reconstructed)

    reconstructed_dataset = pd.DataFrame({'sequence': sequences})
    reconstructed_dataset.index.name = 'index'

    return reconstructed_dataset

In [12]:
save_dataframe_to_one_csv(reconstruct_ordered_events(df), root + 'reconstructed_ordered_events.csv')

In [13]:
reconstructed_dataset = load_reconstructed_events(root + 'reconstructed_ordered_events.csv')

In [14]:
ordered_events_with_durations = reconstructed_dataset.copy()
ordered_events_with_durations['durations'] = df['durations']

In [15]:
ordered_events_with_durations

,index,sequence,durations
0,0,"[88, 38, 38, 45, 45, 45, [44, 45], 45, 45, 38,...","[0.25, 2.5, 0.25, 0.3333, 0.3333, 0.25, 0.3333..."
1,1,"[[56, 65, 68, 80, 60], [54, 61, 65, 75], 77, 7...","[1.25, 1.6667, 0.25, 1.0, 1.0, 0.5, 0.6667, 0...."
2,2,"[46, 70, [53, 60, 62], [65, 46, 53, 60], 70, [...","[2.6667, 0.25, 1.25, 0.75, 0.75, 1.6667, 1.75,..."
3,3,"[[52, 28, 40], 64, [65, 68], [53, 55, 56], [58...","[3.75, 2.5, 1.0, 2.0, 1.6667, 1.3333, 0.75, 0...."
4,4,"[59, 59, [60, 62, 38, 54], 62, [43, 62, 50, 52...","[0.75, 0.6667, 0.6667, 0.5, 1.0, 0.25, 2.0, 0...."
...,...,...,...
2770,2770,"[61, 49, [63, 60], 65, 70, [58, 61, 66, 49], [...","[1.3333, 1.6667, 1.0, 0.5, 0.5, 0.5, 0.6667, 0..."
2771,2771,"[[45, 55, 67, 69, 72, 75], [46, 74, 56, 62, 68...","[1.0, 1.0, 0.25, 1.25, 0.3333, 0.25, 0.25, 0.7..."
2772,2772,"[[45, 57], 71, 52, 61, 64, 45, 69, [45, 69], 4...","[1.6667, 0.6667, 3.3333, 2.75, 2.25, 0.75, 0.5..."
2773,2773,"[48, 41, 60, [67, 68, 72, 79], 66, [65, 68, 72...","[0.3333, 0.3333, 0.25, 0.25, 0.3333, 0.3333, 0..."


In [16]:
mismatches = ordered_events_with_durations.apply(
    lambda row: len(row['sequence']) != len(row['durations']),
    axis=1
)

invalid_rows = ordered_events_with_durations[mismatches]


if not invalid_rows.empty:
    print("Rows with mismatched 'sequence' and 'durations':")
    print(invalid_rows)
else:
    print("All rows have matching 'sequence' and 'durations' lengths.")

All rows have matching 'sequence' and 'durations' lengths.


In [17]:
#save_dataframe_to_one_csv((ordered_events_with_durations), root + 'ordered_events_with_durations.csv')

## Tokenization

In [18]:
class TokenVocabulary:
    def __init__(self, df):
        """
        Build a vocabulary of all unique tokens (note, chord, duration).
        """
        token_set = set()

        for _, row in df.iterrows():
            for event, dur in zip(row['sequence'], row['durations']):
                if isinstance(event, list):
                    note_token = 'CHORD_' + '_'.join(map(str, sorted(event)))
                else:
                    note_token = f'NOTE_{event}'
                dur_token = f'DUR_{dur:.3f}'
                token_set.update([note_token, dur_token])

        self.tokens = sorted(token_set)
        self.token_to_idx = {tok: idx for idx, tok in enumerate(self.tokens)}
        self.idx_to_token = {idx: tok for tok, idx in self.token_to_idx.items()}
        self.vocab_size = len(self.tokens)

    def encode(self, token):
        return self.token_to_idx[token]

    def decode(self, index):
        return self.idx_to_token[index]

    def encode_sequence_and_durations(self, sequence, durations):
        """
        Given a sequence of events and durations, returns a list of token IDs
        in alternating [note_token, dur_token, note_token, dur_token, ...] format.
        """
        tokens = []
        for event, dur in zip(sequence, durations):
            if isinstance(event, list):
                note_token = 'CHORD_' + '_'.join(map(str, sorted(event)))
            else:
                note_token = f'NOTE_{event}'
            dur_token = f'DUR_{dur:.3f}'
            tokens.extend([note_token, dur_token])
        return [self.encode(token) for token in tokens]
    
    def decode_sequence_and_durations(self, token_ids):
        """
        Converts a flat list of token IDs (note/dur alternating) into
        a list of (event, duration) pairs.

        Returns:
            List of tuples: (event, duration), where event is int or list[int], duration is float
        """
        decoded = [self.decode(tid) for tid in token_ids]
        assert len(decoded) % 2 == 0, "Token sequence should be even-length (note-dur pairs)."

        event_sequence = []
        for i in range(0, len(decoded), 2):
            note_token = decoded[i]
            dur_token = decoded[i + 1]

            # Parse note/chord
            if note_token.startswith('NOTE_'):
                event = int(note_token.replace('NOTE_', ''))
            elif note_token.startswith('CHORD_'):
                event = list(map(int, note_token.replace('CHORD_', '').split('_')))
            else:
                raise ValueError(f"Invalid note token: {note_token}")

            # Parse duration
            if dur_token.startswith('DUR_'):
                duration = float(dur_token.replace('DUR_', ''))
            else:
                raise ValueError(f"Invalid duration token: {dur_token}")

            event_sequence.append((event, duration))

        return event_sequence


    

In [19]:
vocab = TokenVocabulary(ordered_events_with_durations)


In [20]:
print("Total tokens:", vocab.vocab_size)
print("Some tokens:", vocab.tokens[:20])  # print first 20 tokens


Total tokens: 154107
Some tokens: ['CHORD_100_100', 'CHORD_100_101', 'CHORD_100_101_101', 'CHORD_100_101_101_102', 'CHORD_100_101_102', 'CHORD_100_101_102_103', 'CHORD_100_101_102_104', 'CHORD_100_101_102_104_105', 'CHORD_100_101_103', 'CHORD_100_101_103_104', 'CHORD_100_101_103_105', 'CHORD_100_101_104', 'CHORD_100_101_105', 'CHORD_100_102', 'CHORD_100_102_103', 'CHORD_100_102_103_104', 'CHORD_100_102_104', 'CHORD_100_102_105', 'CHORD_100_102_107', 'CHORD_100_103']


In [21]:
class TokenizedMusicDataset(Dataset):
    def __init__(self, df, vocab, context_length=20):
        self.samples = []
        self.vocab = vocab
        self.context_length = context_length

        for _, row in df.iterrows():
            sequence = self.vocab.encode_sequence_and_durations(row['sequence'], row['durations'])
            n_tokens = len(sequence)

            # Since each event is 2 tokens, step by 2
            if n_tokens <= 2 * context_length + 2:
                continue

            for i in range(0, n_tokens - 2 * context_length - 2 + 1, 2):
                context = sequence[i : i + 2 * context_length]  # 2 tokens per event
                target = sequence[i + 2 * context_length : i + 2 * context_length + 2]  # next event (2 tokens)
                self.samples.append((context, target))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        context, target = self.samples[idx]
        context_tensor = torch.tensor(context, dtype=torch.long)
        target_tensor = torch.tensor(target, dtype=torch.long)
        return context_tensor, target_tensor



In [22]:
dataset = TokenizedMusicDataset(ordered_events_with_durations, vocab, context_length=20)



In [23]:
x, y = dataset[0]
print("Input token IDs:", x)
print("Target token ID:", y)
print("Decoded input tokens:", [vocab.decode(i.item()) for i in x])
print("Decoded target token:", [vocab.decode(i.item()) for i in y])


Input token IDs: tensor([154095, 153461, 154045, 153608, 154045, 153461, 154052, 153462, 154052,
        153462, 154052, 153461,  51829, 153462, 154052, 153461, 154052, 153462,
        154045, 153467, 112860, 153476,  51829, 153462, 154052, 153461, 154068,
        153470, 154053, 153462, 154053, 153462,  23927, 153479, 154064, 153461,
        154045, 153475, 154053, 153462])
Target token ID: tensor([154045, 153470])
Decoded input tokens: ['NOTE_88', 'DUR_0.250', 'NOTE_38', 'DUR_2.500', 'NOTE_38', 'DUR_0.250', 'NOTE_45', 'DUR_0.333', 'NOTE_45', 'DUR_0.333', 'NOTE_45', 'DUR_0.250', 'CHORD_44_45', 'DUR_0.333', 'NOTE_45', 'DUR_0.250', 'NOTE_45', 'DUR_0.333', 'NOTE_38', 'DUR_0.750', 'CHORD_55_60', 'DUR_1.500', 'CHORD_44_45', 'DUR_0.333', 'NOTE_45', 'DUR_0.250', 'NOTE_61', 'DUR_1.000', 'NOTE_46', 'DUR_0.333', 'NOTE_46', 'DUR_0.333', 'CHORD_38_65', 'DUR_1.750', 'NOTE_57', 'DUR_0.250', 'NOTE_38', 'DUR_1.417', 'NOTE_46', 'DUR_0.333']
Decoded target token: ['NOTE_38', 'DUR_1.000']


In [24]:
import torch
import torch.nn as nn

class MusicTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=2, num_layers=2, dropout=0.1, max_seq_len=512):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(max_seq_len, d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.output_layer = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        """
        x: [batch_size, seq_len] of token indices
        """
        B, T = x.size()
        token_emb = self.token_embedding(x)                         # [B, T, d_model]
        pos_ids = torch.arange(T, device=x.device).unsqueeze(0)     # [1, T]
        pos_emb = self.pos_embedding(pos_ids)                       # [1, T, d_model]

        x = token_emb + pos_emb                                     # [B, T, d_model]
        x = self.transformer(x)                                     # [B, T, d_model]
        logits = self.output_layer(x)                               # [B, T, vocab_size]

        return logits


In [25]:
vocab_size = vocab.vocab_size
model = MusicTransformer(vocab_size=vocab_size)



In [26]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset, batch_size=16, shuffle=True)




In [27]:

import torch.optim as optim

device = torch.device( "cpu")
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [28]:
from tqdm import tqdm

epochs = 1

for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")

    for batch in progress_bar:
        x, y = batch
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        note_logits = logits[:, -2, :]
        dur_logits = logits[:, -1, :]

        loss_note = loss_fn(note_logits, y[:, 0])
        loss_dur = loss_fn(dur_logits, y[:, 1])
        loss = loss_note + loss_dur

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1} complete | Avg Loss: {avg_loss:.4f}")


Epoch 1:   0%|          | 204/278824 [04:22<99:43:32,  1.29s/it, loss=15.8] 


KeyboardInterrupt: 